In [185]:
import numpy as np
import pandas as pd
import datetime 
import math

## 



## Tables input

In [2]:
#Tables input
name_block    = 'Observing_blocks.csv'
name_modes    = 'Observing_modes.csv'
name_timeline = 'Timeline_blocks.csv'

In [214]:
# Read tables
timeline = pd.read_csv(name_timeline)
modes    = pd.read_csv(name_modes)
blocks   = pd.read_csv(name_block)

In [213]:
def generate_initial_tables():
  #Define outputs
  colunms = {'sunrise':['OBS_ID','BLOCK','START_DATE','START_TIME','END_DATE','END_TIME', 'XCEN','YCEN','OBS_MODES','ROI','COMMENT'],
            'susi':['OBS_ID','BLOCK','Instrument','START_DATE','START_TIME','END_DATE','END_TIME', 'XCEN','YCEN','FOV','OBS_MODES','INTRUMENT_MODE','ROI','COMMENT'],
            'scip':['OBS_ID','BLOCK','Instrument','START_DATE','START_TIME','END_DATE','END_TIME', 'XCEN','YCEN','FOV','OBS_MODES','INTRUMENT_MODE','ROI','COMMENT'],
            'tumag':['OBS_ID','BLOCK','Instrument','START_DATE','START_TIME','END_DATE','END_TIME', 'XCEN','YCEN'      ,'OBS_MODES','INTRUMENT_MODE','ROI','COMMENT']}

  Sunrise =   pd.DataFrame(columns=colunms['sunrise'])
  Susi    =   pd.DataFrame(columns=colunms['susi'])
  Scip    =   pd.DataFrame(columns=colunms['scip'])
  TuMag   =   pd.DataFrame(columns=colunms['tumag'])
  return Sunrise, Susi, Scip, TuMag


In [219]:
def add_var(ll,obs_mode,modes,timeline,current_time,Slit_inst,name):  
  #print('-----> ',name)
  var_slit = list(timeline.loc[ll,['OBS_ID','BLOCK','BLOCK','START_DATE','START_TIME','END_DATE','END_TIME','X_TUMAG','Y_TUMAG']])
  var_slit[Slit_inst.columns.get_loc('Instrument')] =name
  
  if name != 'TUMAG': 
    var_slit[Slit_inst.columns.get_loc('XCEN')]=float(var_slit[Slit_inst.columns.get_loc('XCEN')])+float(timeline.loc[ll,'X_OFF_{}'.format(name)])
    fov = modes['{}_FOV'.format(name)].loc[np.where(modes['OBSERVING_MODE'] == obs_mode)[0][0]]
    if math.isnan(fov): fov='-'
    var_slit.append(fov)

  var_slit.append(obs_mode)
  var_slit.append(modes['{}_MODE'.format(name)].loc[np.where(modes['OBSERVING_MODE'] == obs_mode)[0][0]])
  var_slit.append(modes['Target'].loc[np.where(modes['OBSERVING_MODE'] == obs_mode)[0][0]])
  var_slit.append(timeline.loc[ll,['COMMENT']][0])

  ### UPDATE START TIMES
  var_slit[Slit_inst.columns.get_loc('START_DATE')]=datetime.datetime.strftime(current_time,"%d/%m/%Y")
  var_slit[Slit_inst.columns.get_loc('START_TIME')]=datetime.datetime.strftime(current_time,"%H:%M:%S")

  # add the duration of the obs 
  delta_min=modes['{}_DURATION'.format(name)].loc[np.where(modes['OBSERVING_MODE'] == obs_mode)[0][0]]
  if math.isnan(delta_min): delta_min=0
  delta_time = datetime.timedelta(minutes=delta_min)
  current_time=current_time+delta_time

  var_slit[Slit_inst.columns.get_loc('END_DATE')]=datetime.datetime.strftime(current_time,"%d/%m/%Y")
  var_slit[Slit_inst.columns.get_loc('END_TIME')]=datetime.datetime.strftime(current_time,"%H:%M:%S")
  #print(var_slit)
  #print('SIZE ===>',name, " : ",np.shape(var_slit)[0],' --',Slit_inst.shape)
  Slit_inst.loc[len(Slit_inst)]=var_slit
  
  return Slit_inst,current_time


In [220]:
# Generate tables

Sunrise, Susi, Scip, TuMag = generate_initial_tables()


for ll in range(timeline.shape[0]):
  #Time variables to control the times for each obs mode
  s_time=np.array(timeline.loc[ll,['START_DATE','START_TIME']])
  current_time= datetime.datetime(year=int(s_time[0][6:]),month=int(s_time[0][3:5]),day=int(s_time[0][0:2]),
          hour=int(s_time[1][:2]),minute=int(s_time[1][3:5]),second=int(s_time[1][6:]))


  #Find the block
  loc_block=np.where(blocks['BLOCK'] == timeline['BLOCK'][ll])[0][0]

  n_mode_loc=blocks.columns.get_loc('OBS1')
  tmp_obs,tmp_target=[],[]

    # Other instruments


  for j in range(n_mode_loc,n_mode_loc+blocks.loc[loc_block][2]):
    obs_mode =blocks.loc[loc_block][j]
    
    if obs_mode != 'NAN':
      print(obs_mode)
      tmp_obs.append(obs_mode)


      Susi, current_time= add_var(ll,obs_mode,modes,timeline,current_time,Susi,'SUSI')
      Scip, current_tmp= add_var(ll,obs_mode,modes,timeline,current_time,Scip,'SCIP')
      TuMag, current_tmp= add_var(ll,obs_mode,modes,timeline,current_time,TuMag,'TUMAG')
      


  # General coords
  var = list(timeline.loc[ll,['OBS_ID','BLOCK','START_DATE','START_TIME','END_DATE','END_TIME','X_TUMAG','Y_TUMAG']])
  var.append(', '.join(map(str, tmp_obs)))
  var.append(modes['Target'].loc[np.where(modes['OBSERVING_MODE'] == obs_mode)[0][0]])
  var.append(timeline.loc[ll,['COMMENT']][0])
  Sunrise.loc[len(Sunrise)]=var

Sunrise.to_csv('Timeline_MAIN_page.csv')
Susi.to_csv('Timeline_SUSI_page.csv')
Scip.to_csv('Timeline_SCIP_page.csv')
TuMag.to_csv('Timeline_TUMAG_page.csv')



CAL0
QS1
QS2
QS4a
QS4b
QS8
CAL1
QS6
QS3
QS5a
CAL1
QS5b
QS7c
CAL1
FS1
CAL1
FS2
CAL0
AR1
AR2
CAL1
AR5a
AR5b
AR3
CAL1
AR4c
AR4b
AR4a
CAL1
SP1
SP2
SP4a
SP4b
CAL1
SP3
SP8
CAL1
SP3
SP8
CAL1
SP3
SP8
CAL1
SP7a
SP7b
CAL1
SP6
CAL0
AR6a
AR6b
AR6c
CAL1
AR7
CAL1
PL1
PL4
PL5
CAL1
PL2
PL3
CAL1
EW1
EW2
EW3
CAL1
CH1
CH2
CH3
CAL0
Flare1
CAL1
QS1
QS2
QS4a
QS4b
QS8
CAL1
QS6
QS3
QS5a
CAL1
QS5b
QS7c


In [203]:
Sunrise

,OBS_ID,BLOCK,START_DATE,START_TIME,END_DATE,END_TIME,XCEN,YCEN,OBS_MODES,ROI
0,OB_09052022_001000,CALIB_0,09/05/2022,10:00:00,09/05/2022,16:00:00,NaN,NaN,CAL0,Comissioning
1,OB_09052022_001600,QSDC_1,09/05/2022,16:05:00,09/05/2022,22:09:00,-90.0,-245.0,"QS1, QS2, QS4a, QS4b, QS8",Quiet Sun
2,OB_09052022_002200,CALIB_1,09/05/2022,22:14:00,09/05/2022,22:44:00,NaN,NaN,CAL1,Calibration
3,OB_09052022_002200,QSDC_2,09/05/2022,22:14:00,10/05/2022,03:01:00,212.0,-427.0,"QS6, QS3, QS5a",Quiet Sun
4,OB_10052022_000300,CALIB_1,10/05/2022,03:06:00,10/05/2022,03:36:00,NaN,NaN,CAL1,Calibration
...,...,...,...,...,...,...,...,...,...,...
128,OB_13052022_002300,QSDC_1,13/05/2022,23:56:00,14/05/2022,06:00:00,-524.0,-238.0,"QS1, QS2, QS4a, QS4b, QS8",Quiet Sun
129,OB_14052022_000600,CALIB_1,14/05/2022,06:05:00,14/05/2022,06:35:00,NaN,NaN,CAL1,Calibration
130,OB_14052022_000600,QSDC_2,14/05/2022,06:40:00,14/05/2022,11:27:00,456.0,-475.0,"QS6, QS3, QS5a",Quiet Sun
131,OB_14052022_001100,CALIB_1,14/05/2022,11:32:00,14/05/2022,12:02:00,NaN,NaN,CAL1,Calibration


In [80]:
var_susi[Susi.columns.get_loc('START_DATE')]=datetime.datetime.strftime(new_time,"%d/%m/%Y")
var_susi[Susi.columns.get_loc('START_TIME')]=datetime.datetime.strftime(new_time,"%H:%M:%S")


In [217]:
timeline.loc[ll,['COMMENT']][0]

nan

In [178]:
timeline.shape[0]

48

In [56]:
s_time=np.array(timeline.loc[ll,['START_DATE','START_TIME']])

current_time= datetime.datetime(year=int(s_time[0][6:]),month=int(s_time[0][0:2]),day=int(s_time[0][3:5]),
         hour=int(s_time[1][:2]),minute=int(s_time[1][3:5]),second=int(s_time[1][6:]))

delta_time = datetime.timedelta(minutes=30)

new_time=current_time+delta_time

print(datetime.datetime.strftime(new_time,"%d/%m/%YT%H:%M:%S"),
datetime.datetime.strftime(current_time,"%d/%m/%YT%H:%M:%S")
)

07/04/2022T06:35:00 07/04/2022T06:05:00


In [ ]:
timeline['Block'][]

'QSDC_1'

In [86]:
Susi, current_time= add_var(var_susi,obs_mode,modes,current_time,Susi)

ValueError: cannot set a row with mismatched columns